In [2]:
import findspark
findspark.init()

In [3]:
import pyspark

In [3]:
import sys
print(sys.executable)

/usr/bin/python3


In [4]:
pyspark.__version__

'3.3.3'

In [5]:
pyspark.__file__

'/opt/spark/python/pyspark/__init__.py'

In [6]:
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession.builder\
        .config("spark.sql.execution.pythonUDF.arrow.enabled", "false") \
        .master("local[*]")\
        .appName('test')\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/28 13:41:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
!head taxi+_zone_lookup.csv

"LocationID","Borough","Zone","service_zone"
1,"EWR","Newark Airport","EWR"
2,"Queens","Jamaica Bay","Boro Zone"
3,"Bronx","Allerton/Pelham Gardens","Boro Zone"
4,"Manhattan","Alphabet City","Yellow Zone"
5,"Staten Island","Arden Heights","Boro Zone"
6,"Staten Island","Arrochar/Fort Wadsworth","Boro Zone"
7,"Queens","Astoria","Boro Zone"
8,"Queens","Astoria Park","Boro Zone"
9,"Queens","Auburndale","Boro Zone"


In [7]:
df = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

In [ ]:
df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [ ]:
df.write.parquet('zones')

AnalysisException: path file:/home/devmarrie/Desktop/coding/DE_learning/DataEngineering/data_talks_camp/week5/zones already exists.

In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet

--2023-09-25 21:09:04--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.160.203.53, 3.160.203.81, 3.160.203.184, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.160.203.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308924937 (295M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2021-01.parquet’

fhvhv_tripdata_2021 100%[===================>] 294.61M  1.60MB/s    in 3m 26s  

2023-09-25 21:12:32 (1.43 MB/s) - ‘fhvhv_tripdata_2021-01.parquet’ saved [308924937/308924937]



In [ ]:
!wc -l fhvhv_tripdata_2021-01.parquet

1006794 fhvhv_tripdata_2021-01.parquet


In [8]:
# read data from the csv file
df2 = spark.read \
    .option("header", "true") \
    .parquet('fhvhv_tripdata_2021-01.parquet')

In [9]:
df2.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [10]:
df2 # contain their data type as well

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, originating_base_num: string, request_datetime: timestamp, on_scene_datetime: timestamp, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: bigint, DOLocationID: bigint, trip_miles: double, trip_time: bigint, base_passenger_fare: double, tolls: double, bcf: double, sales_tax: double, congestion_surcharge: double, airport_fee: double, tips: double, driver_pay: double, shared_request_flag: string, shared_match_flag: string, access_a_ride_flag: string, wav_request_flag: string, wav_match_flag: string]

In [11]:
df2.head()

Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime=datetime.datetime(2021, 1, 1, 3, 28, 9), on_scene_datetime=datetime.datetime(2021, 1, 1, 3, 31, 42), pickup_datetime=datetime.datetime(2021, 1, 1, 3, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 3, 49, 7), PULocationID=230, DOLocationID=166, trip_miles=5.26, trip_time=923, base_passenger_fare=22.28, tolls=0.0, bcf=0.67, sales_tax=1.98, congestion_surcharge=2.75, airport_fee=None, tips=0.0, driver_pay=14.99, shared_request_flag='N', shared_match_flag='N', access_a_ride_flag=' ', wav_request_flag='N', wav_match_flag='N')

In [10]:
#!head -n 1001 fhvhv_tripdata_2021-01.parquet > head.parquet(cannot work for parquet since it is a binary file) works for csv
df_head = df2.limit(1001)

In [11]:
# Subdivide the data to have no idle excecutor write to a new file after finishing to see how expensive it is
df_head = df_head.repartition(24)

In [14]:
#write it to parquet file
df_head.write.parquet("head_1000.parquet")

AnalysisException: path file:/home/devmarrie/Desktop/coding/DE_learning/DataEngineering/data_talks_camp/week5/head_1000.parquet already exists.

In [12]:
df_head.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [16]:
df_head

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, originating_base_num: string, request_datetime: timestamp, on_scene_datetime: timestamp, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: bigint, DOLocationID: bigint, trip_miles: double, trip_time: bigint, base_passenger_fare: double, tolls: double, bcf: double, sales_tax: double, congestion_surcharge: double, airport_fee: double, tips: double, driver_pay: double, shared_request_flag: string, shared_match_flag: string, access_a_ride_flag: string, wav_request_flag: string, wav_match_flag: string]

In [17]:
df_head.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampType(), True), StructField('on_scene_datetime', TimestampType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_request_flag',

In [18]:
df_head.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nul

In [13]:
df_head.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID', 'dispatching_base_num') \
       .filter(df_head.hvfhs_license_num == 'HV0003') \
       .show()

+-------------------+-------------------+------------+------------+--------------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|dispatching_base_num|
+-------------------+-------------------+------------+------------+--------------------+
|2021-01-01 03:29:05|2021-01-01 03:45:21|         141|           7|              B02877|
|2021-01-01 03:25:59|2021-01-01 03:32:32|         242|         248|              B02682|
|2021-01-01 03:05:24|2021-01-01 03:10:46|          26|         227|              B02835|
|2021-01-01 03:17:32|2021-01-01 03:34:03|         255|         225|              B02880|
|2021-01-01 03:33:48|2021-01-01 03:45:38|         213|         126|              B02764|
|2021-01-01 03:46:05|2021-01-01 04:03:30|          36|          62|              B02617|
|2021-01-01 03:17:29|2021-01-01 03:32:53|          37|          72|              B02764|
|2021-01-01 03:56:02|2021-01-01 04:05:31|          68|         137|              B02867|
|2021-01-01 03:18:42|

In [14]:
from pyspark.sql import functions as F

In [15]:
# inbuilt functions
df_head \
   .withColumn('pickup_date', F.to_date(df_head.pickup_datetime)) \
   .withColumn('dropoff_date', F.to_date(df_head.dropoff_datetime)) \
   .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
   .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-01|  2021-01-01|          47|         127|
| 2021-01-01|  2021-01-01|          68|         112|
| 2021-01-01|  2021-01-01|         170|         129|
| 2021-01-01|  2021-01-01|          69|         244|
| 2021-01-01|  2021-01-01|         241|         241|
| 2021-01-01|  2021-01-01|          48|          97|
| 2021-01-01|  2021-01-01|          80|          90|
| 2021-01-01|  2021-01-01|           4|          49|
| 2021-01-01|  2021-01-01|         119|         159|
| 2021-01-01|  2021-01-01|         211|         223|
| 2021-01-01|  2021-01-01|          61|         177|
| 2021-01-01|  2021-01-01|         140|         173|
| 2021-01-01|  2021-01-01|         106|         109|
| 2021-01-01|  2021-01-01|          68|         114|
| 2021-01-01|  2021-01-01|         179|          37|
| 2021-01-01|  2021-01-01|         212|       

In [16]:
# user defined functions
def crazy_stuff_fnc(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [17]:
crazy_stuff_fnc('B02510')

'e/9ce'

In [20]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
crazy_stuff_udf = udf(crazy_stuff_fnc, StringType())

In [25]:
df_head

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, originating_base_num: string, request_datetime: timestamp, on_scene_datetime: timestamp, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: bigint, DOLocationID: bigint, trip_miles: double, trip_time: bigint, base_passenger_fare: double, tolls: double, bcf: double, sales_tax: double, congestion_surcharge: double, airport_fee: double, tips: double, driver_pay: double, shared_request_flag: string, shared_match_flag: string, access_a_ride_flag: string, wav_request_flag: string, wav_match_flag: string]

In [21]:
df_head \
   .withColumn('pickup_date', F.to_date(df_head.pickup_datetime)) \
   .withColumn('dropoff_date', F.to_date(df_head.dropoff_datetime)) \
   .withColumn('base_id', crazy_stuff_udf(df_head['dispatching_base_num'])) \
   .select('pickup_date', 'dropoff_date','base_id', 'PULocationID', 'DOLocationID') \
   .show()

Traceback (most recent call last):
  File "/opt/spark/python/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/opt/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 692, in reducer_override
    return self._function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 565, in _function_reduce
    return self._dynamic_function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 546, in _dynamic_function_reduce
    state = _function_getstate(func)
            ^^^^^^^^^^^^^^^

PicklingError: Could not serialize object: IndexError: tuple index out of range